<a href="https://colab.research.google.com/github/kavyajeetbora/foursquare_ai/blob/master/notebooks/15.railways.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here is the link to extract more information on railway stations:

https://www.railyatri.in/stations?name=L&page=10

In [1]:
import duckdb

In [2]:
%%shell

wget https://download.geofabrik.de/asia/india-latest.osm.pbf -O india-latest.osm.pbf

--2025-12-18 16:44:54--  https://download.geofabrik.de/asia/india-latest.osm.pbf
Resolving download.geofabrik.de (download.geofabrik.de)... 95.216.245.185, 95.216.245.233, 95.217.45.61, ...
Connecting to download.geofabrik.de (download.geofabrik.de)|95.216.245.185|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://download.geofabrik.de/asia/india-251217.osm.pbf [following]
--2025-12-18 16:44:54--  https://download.geofabrik.de/asia/india-251217.osm.pbf
Reusing existing connection to download.geofabrik.de:443.
HTTP request sent, awaiting response... 200 OK
Length: 1640244337 (1.5G) [application/octet-stream]
Saving to: ‘india-latest.osm.pbf’

india-latest.osm.pb 100%[===================>]   1.53G  13.0MB/s    in 2m 8s   

2025-12-18 16:47:03 (12.2 MB/s) - ‘india-latest.osm.pbf’ saved [1640244337/1640244337]



In [33]:
con = duckdb.connect()

con.execute("INSTALL httpfs;")
con.execute("LOAD httpfs;")
con.execute("INSTALL spatial;")
con.execute("LOAD spatial;")

query = f"""
SELECT
    id,
    unnest(map_extract(tags, 'name'))              AS name,
    unnest(map_extract(tags, 'railway'))           AS railway,
    unnest(map_extract(tags, 'public_transport'))  AS public_transport,
    unnest(map_extract(tags, 'ref')) AS station_code,
    unnest(map_extract(tags, 'network')) AS network,
    unnest(map_extract(tags, 'internet_access')) AS internet_access,
    lat,
    lon
FROM ST_READOSM('india-latest.osm.pbf')
WHERE kind = 'node'
AND 'station' IN map_extract(tags, 'railway')
AND 'station' IN map_extract(tags, 'public_transport')
"""

df = con.execute(query).df()
con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [29]:
df.head()

,id,name,railway,public_transport,station_code,network,internet_access,lat,lon
0,30518283,Vasai Road,station,station,BSR,Mumbai Suburban Railway,no,19.382668,72.832025
1,30518292,Naigaon,station,station,NIG,Mumbai Suburban Railway,no,19.351093,72.846523
2,30518297,Bhayandar,station,station,BYR,Mumbai Suburban Railway,wlan,19.310185,72.852699
3,30518361,Kharbhav,station,station,KHBV,IR,no,19.298321,72.984468
4,30519796,Ranjangaon Road,station,station,RNJD,IR,no,18.878166,74.606712


In [30]:
df.shape

(7300, 9)